## 优化投资组合

所谓的优化投资组合其实指的是，针对某个评价指标，找到投资产品之间最优的比例。

下面代码，以夏普比例为优化目标，找到最佳的投资组合。需要注意的是，组合系数是有约束条件的；优化器是求最小值，而夏普比率需要最大化。

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo
import tushare as ts


def get_stocks_data(stocks_name_code, start_date, end_date):
    '''
    根据股票名称列表，获取股票信息
    参数：
        names:     名称列表
        start_date:开始时间
        end_date:  结束时间
    返回：股票组合DataFrame
    '''
    # 创建一个时间的空dataframe
    dates = pd.date_range(start_date, end_date)
    stocks = pd.DataFrame(index=dates)
    for name in stocks_name_code.keys():
        data = ts.get_k_data(stocks_name_code[name], start=start_date, end=end_date)
        data = data.set_index(['date'])
        data = data[['close']]
        data = data.rename(columns={'close': name})
        stocks = stocks.join(data, how='inner')

    return stocks


def get_sharpe_ratio(allocs, stocks, start_value=1e5, K=1, daily_risk_free=0):
    '''
    由每日收益列表得到组合统计变量，对参数的顺序是由要求的，求解参数在前
    参数：
        allocs:      资金比例
        stocks:      股票组合DataFrame
        start_value: 开始金额
        pos_values：每日资金金额列表
        K：采样频率系数
        daily_risk_free: 无风险收益
    '''
    #### 先得到日组合投资值 ######
    # 归一化
    normed_stocks = stocks / stocks.iloc[0, :]
    # 计算每日资金额
    pos_values = normed_stocks * allocs * start_value
    pos_values = pos_values.sum(axis=1)

    cumulative_return = (pos_values[-1] / pos_values[0]) - 1
    # 第一天的收益实际为0，去掉第一天的收益
    daily_returns = (pos_values[1:] / pos_values.values[:-1]) - 1
    avg_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()
    sharpe_ratio = K * (avg_daily_return - daily_risk_free) / std_daily_return

    # 优化器是求最小化的，通过-1来求最大化
    return -1.0 * sharpe_ratio


def conditions(min=0.0, max=1.0, sum=1.0):
    '''
    约束条件函数，约束条件分为eq 和ineq。
    eq表示函数结果等于0； ineq表示表达式大于等于0
    '''
    cons = ({'type': 'ineq', 'fun': lambda x: x - min}, \
            {'type': 'ineq', 'fun': lambda x: -x + max}, \
            {'type': 'eq', 'fun': lambda x: np.sum(x) - sum},)
    return cons


def main():
    # 股票名称和代码的映射字典
    stocks_name_code = {
        # 'hs300' : 'hs300',
        'zgpa': '601318',  # 中国平安
        'zgsh': '600028',  # 中国石化
        'gzmt': '600519',  # 贵州茅台
        'nfhk': '600029',  # 南方航空
    }
    start_date = '2012-01-01'
    end_date = '2012-12-31'
    stocks = get_stocks_data(stocks_name_code, start_date, end_date)

    # 资金比例初始值
    allocs_init = np.array([0.4, 0.4, 0.1, 0.1])

    # 设置约束条件
    cons = conditions()

    # 调用优化器求解
    result = spo.minimize(get_sharpe_ratio, allocs_init, args=(stocks,), method='SLSQP', options={'disp': True},
                          constraints=cons)
    print(result.x)
    print(-result.fun)


if __name__ == "__main__":
    main()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.07765767611919824
            Iterations: 9
            Function evaluations: 54
            Gradient evaluations: 9
[ 8.74981329e-01  4.98732999e-18  1.25018671e-01 -3.51281504e-17]
0.07765767611919824
